In [1]:
LANGHCHAIN_TRACINB_V2 = "true"
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [2]:
import json

with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model='openchat')

In [4]:
llm.invoke("hello")

' Hello! How can I help you today? If you have any questions or need assistance, please feel free to ask.'

In [12]:
import joblib

top_preds = joblib.load('./vecs/top_preds')
true_labels = joblib.load('./vecs/true_labels')
doc_labels = joblib.load('./vecs/doc_label')

In [39]:
len(top_preds), len(true_labels), len(dataset[0]['annotations'][0]['result'])

(91, 91, 91)

In [19]:
for p in top_preds:
    if len(p)!=5:
        print(len(p))

In [27]:
curr_sent = dataset[0]['annotations'][0]['result'][0]['value']['text']
curr_sent

'      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013'

In [31]:
prompt = f'''You are a specialized system focused on sentence classification of court opinion.
        RHETORICAL ROLE:
        Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
        a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
        roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
        conclusions, each contributing to the document's overall persuasive and informative structure.
        Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
        'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
        'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
        The definition of each rhetorical role is given below:
        Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
        Facts: 	This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.). Depositions and proceedings of current court. Summary of lower court proceedings.
        Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
        Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES.
        Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
        Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
        Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS
        Statute : Text in which the court discusses Established laws, which can come from a mixture of sources – Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute
        Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent
        Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.
        Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum.
        Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale
        NONE: If a sentence does not belong to any of the above categories.
        
        You are given top 5 most similar sentences and their corresponding labels:

        '''

for p in top_preds[0]:
    prompt+= f"SENTENCE: {p[0].strip()}" + f"\nLABEL: {p[1]}\n"

pred_prompt = f'''Your task is to predict the label of the current sentence using the previous provided most similar sentences.
Your output should only be the label of the current sentence as a string and nothing else.
Now do prediction for this current sentence: {curr_sent.strip()}'''

prompt += pred_prompt
        

In [32]:
print(prompt)

You are a specialized system focused on sentence classification of court opinion.
        RHETORICAL ROLE:
        Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
        a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
        roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
        conclusions, each contributing to the document's overall persuasive and informative structure.
        Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
        'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
        'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
        The definition of each rhetorical role is g

In [35]:
response = llm.invoke(prompt).strip()

In [36]:
response

'Preamble'

In [38]:
from tqdm import tqdm

In [43]:
def run_preds(data):
    preds = []
    for i in tqdm(range(0, len(data))):
        curr_sent = dataset[0]['annotations'][0]['result'][i]['value']['text']
        

        prompt = f'''You are a specialized system focused on sentence classification of court opinion.
                RHETORICAL ROLE:
                Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
                a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
                roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
                conclusions, each contributing to the document's overall persuasive and informative structure.
                Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
                'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
                'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
                The definition of each rhetorical role is given below:
                Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
                Facts: 	This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.). Depositions and proceedings of current court. Summary of lower court proceedings.
                Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
                Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES.
                Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
                Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
                Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS
                Statute : Text in which the court discusses Established laws, which can come from a mixture of sources – Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute
                Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent
                Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.
                Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum.
                Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale
                NONE: If a sentence does not belong to any of the above categories.
                
                You are given top 5 most similar sentences and their corresponding labels:

                '''

        for p in top_preds[i]:
            prompt+= f"SENTENCE: {p[0].strip()}" + f"\nLABEL: {p[1]}\n"

        pred_prompt = f'''Your task is to predict the label of the current sentence.
        You should use the most similar sentences and their labels to predict the label of the current sentence.
        Your output should only be the label of the current sentence as a string and nothing else.
        Now do prediction for this current sentence:
        {curr_sent.strip()}'''

        prompt += pred_prompt
    
        response = llm.invoke(prompt).strip()
        preds.append(response)
    
    return preds
    

In [44]:
llm_labels = run_preds(dataset[0]['annotations'][0]['result'])
len(llm_labels)

100%|██████████| 91/91 [00:46<00:00,  1.97it/s]


91

In [49]:
llm_labels[-10:], true_labels[-10:]

(['ANALYSIS',
  'ANALYSIS',
  'ANALYSIS',
  'ANALYSIS',
  'Ruling by Present Court',
  'RPC',
  'Ruling by Present Court',
  'RPC',
  'RPC',
  'NONE'],
 ['ANALYSIS',
  'ANALYSIS',
  'RATIO',
  'ANALYSIS',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'NONE'])

In [56]:
label_abbrv = {
    'preamble': 'PREAMBLE',
    'facts': 'FAC',
    'fac': 'FAC',
    'ruling by lower court': 'RLC',
    'rlc': 'RLC',
    'issues': 'ISSUE',
    'issue': 'ISSUE',
    'argument by petitioner': 'ARG_PETITIONER',
    'arg_petitioner': 'ARG_PETITIONER',
    'argument by respondent': 'ARG_RESPONDENT',
    'arg_respondent': 'ARG_RESPONDENT',
    'analysis': 'ANALYSIS',
    'statute ': 'STA',
    'precedent relied': 'PRE_RELIED',
    'pre_relied': 'PRE_RELIED',
    'precedent not relied': 'PRE_NOT_RELIED',
    'pre_not_relied': 'PRE_NOT_RELIED',
    'ratio of the decision': 'Ratio',
    'ruling by present court': 'RPC',
    'rpc': 'RPC',
    'none': 'NONE'
}
  
abbrv_label = {v: k for k, v in label_abbrv.items()}
abbrv_label

{'PREAMBLE': 'preamble',
 'FAC': 'fac',
 'RLC': 'rlc',
 'ISSUE': 'issue',
 'ARG_PETITIONER': 'arg_petitioner',
 'ARG_RESPONDENT': 'arg_respondent',
 'ANALYSIS': 'analysis',
 'STA': 'statute ',
 'PRE_RELIED': 'pre_relied',
 'PRE_NOT_RELIED': 'pre_not_relied',
 'Ratio': 'ratio of the decision',
 'RPC': 'rpc',
 'NONE': 'none'}

In [59]:
preds = []
trues = []
for i, label in enumerate(llm_labels):
    try:
        preds.append(label_abbrv[label.lower()])
        trues.append(true_labels[i])
    except:
        print(i, label.lower())

64 pre_ana


In [60]:
len(preds), len(trues)

(90, 90)

In [63]:
preds[:10], trues[:10]

(['PREAMBLE',
  'PREAMBLE',
  'FAC',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'ANALYSIS',
  'FAC'],
 ['PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'])

In [48]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [64]:
acc = accuracy_score(trues, preds)
f1 = f1_score(trues, preds, average="macro")

In [65]:
acc, f1

(0.35555555555555557, 0.3746699207225523)

In [67]:
print(classification_report(trues, preds))

                precision    recall  f1-score   support

      ANALYSIS       0.17      0.67      0.27         9
ARG_PETITIONER       0.33      0.27      0.30        11
ARG_RESPONDENT       0.50      0.06      0.11        17
           FAC       0.39      0.75      0.51        12
         ISSUE       0.00      0.00      0.00         0
          NONE       0.67      1.00      0.80         2
      PREAMBLE       1.00      0.75      0.86         4
PRE_NOT_RELIED       0.00      0.00      0.00         0
    PRE_RELIED       1.00      0.04      0.08        25
         RATIO       0.00      0.00      0.00         1
           RLC       1.00      0.50      0.67         4
           RPC       0.83      1.00      0.91         5

      accuracy                           0.36        90
     macro avg       0.49      0.42      0.37        90
  weighted avg       0.63      0.36      0.31        90



/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 